In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

import os
import numpy as np
from pandas import DataFrame
import pandas as pd
import re
from notebook_utils import NotebookUtilities
from datetime import timedelta
import matplotlib.pyplot as plt
import re

nu = NotebookUtilities(data_folder_path=os.path.abspath('../data'))

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(842663, 113)


In [22]:

nu.conjunctify_nouns(largest_group_df.action_type.unique().tolist())

'INJURY_TREATED, TOOL_APPLIED, PATIENT_ENGAGED, and PATIENT_DEMOTED'

In [26]:

# Of the simultaneous actions:
#   1. PATIENT_RECORD and INJURY_RECORD together spam every scene at the same time.
#   2. S_A_L_T_WALK_IF_CAN instantly causes numerous S_A_L_T_WALKED loggings.
#   3. Similarly with S_A_L_T_WAVE_IF_CAN and S_A_L_T_WAVED.
#   4. Lots of VOICE_CAPTUREs create instantaneous VOICE_COMMANDs.
#   5. And the logs are hit regularly with PLAYER_GAZE and PLAYER_LOCATION,
#      which appear at the same elapsed_time as whatever was happening at the time
#      (and usually instantaneous to each other).
#   6. Also, INJURY_TREATED sometimes causes an instantaneous PATIENT_DEMOTED.
#      So you can have INJURY_TREATED, TOOL_APPLIED, PATIENT_ENGAGED, and
#      PATIENT_DEMOTED all at the same time.
simultaneous_actions_list = [
    'PATIENT_RECORD', 'INJURY_RECORD', 'S_A_L_T_WALK_IF_CAN', 'S_A_L_T_WALKED', 'S_A_L_T_WAVED', 'S_A_L_T_WAVE_IF_CAN', 'VOICE_COMMAND', 'PLAYER_GAZE', 'PLAYER_LOCATION',
    # 'PATIENT_DEMOTED'
]

# Create a groupby object
mask_series = ~frvrs_logs_df.action_type.isin(simultaneous_actions_list)
groupby_columns = ['session_uuid', 'scene_index', 'elapsed_time']
gb = frvrs_logs_df[mask_series].groupby(groupby_columns)

# Get the group sizes
group_sizes = gb.size()

# Get the index of the group with the largest size
largest_group_index = group_sizes.idxmax()

# Get the data frame for the group with the largest size
largest_group_df = gb.get_group(largest_group_index)

# Print the data frame with the largest row count
print(largest_group_df.shape)
print(largest_group_df.action_type.unique().tolist())
df = largest_group_df.sample(4).dropna(axis='columns', how='all')
display(df.T)

(4, 113)
['INJURY_TREATED', 'TOOL_APPLIED', 'PATIENT_ENGAGED', 'PATIENT_DEMOTED']


,143475,143478,143477,143476
action_type,INJURY_TREATED,PATIENT_DEMOTED,PATIENT_ENGAGED,TOOL_APPLIED
elapsed_time,70727,70727,70727,70727
event_time,2022-12-06 14:06:26,2022-12-06 14:06:26,2022-12-06 14:06:26,2022-12-06 14:06:26
session_uuid,2e8678fd-fdc4-40c2-8122-09f1e31b8d2a,2e8678fd-fdc4-40c2-8122-09f1e31b8d2a,2e8678fd-fdc4-40c2-8122-09f1e31b8d2a,2e8678fd-fdc4-40c2-8122-09f1e31b8d2a
file_name,All CSV files renamed by date/12.06.22.1410.csv,All CSV files renamed by date/12.06.22.1410.csv,All CSV files renamed by date/12.06.22.1410.csv,All CSV files renamed by date/12.06.22.1410.csv
logger_version,1.0,1.0,1.0,1.0
scene_index,0,0,0,0
patient_demoted_health_level,NaN,100,NaN,NaN
patient_demoted_health_time_remaining,NaN,Infinity,NaN,NaN
patient_demoted_id,NaN,Mike_9 Root,NaN,NaN


In [27]:

file_name = largest_group_df.file_name.unique().item()
renamed_file = fu.get_new_file_name(file_name)
print(f'Look in {file_name} (or {renamed_file}) for {largest_group_index[-1]} in the second column.')

Look in All CSV files renamed by date/12.06.22.1410.csv (or All CSV files renamed by date/22.12.06.1405.csv) for 70727 in the second column.
